# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later
import sys
import os
import json
import pandas as pd
sys.path.insert(1, '../../../../src/')
import gt_modules.evaluation as evaluation
from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-m5-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Sherlock Holmes explorative search") 


Consider the following exploratory scenario:


> Investigate the canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes) and which are the films and tv series related to the canon. We are also interested in the Emmy Awards and hiw many awards each film won. 


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q35610` | Arthur Conan Doyle        | node |



Also consider

```
wd:Q35610 ?p ?obj .
```

is the BGP to retrieve all **properties of Arthur Conan Doyle**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes)

2. Identify the BGP for film

3. Identify the BGP for television series

4. Return the year of publication (take the first one if there are more publication dates) for each film and television series related to the canon of Sherlock Holmes (the result set must be a list of triples film/tv series IRI, label and year).

5. Find who acted in more than one film or tv series related to the canon of Sherlock Holmes (the result set must be a list of triples actor/actress IRI, label and #film).

6. Identify the BGP for Primetime Emmy Award

7. Return all the types of Primetime Emmy Award (the result set must be a list of couples award IRI and label).

8. For each film and television series related to the canon of Sherlock Holmes, return the number of Primetime Emmy Awards they won (return also the objects who did not win an Award) (the result set must be a list of triples film/tv series IRI, label and #award).

In [2]:
## startup the evaluation
# setup the file and create the empty json
ipname = "m5.ipynb"
pt = os.getcwd()+os.sep+ipname
evaluation.setup(pt)

The index of this workflow is: 2_1


## Task 1

In [3]:
# object properties of Arthur Conan Doyle
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 

    wd:Q35610 ?p ?o.
    ?p sc:name ?pName .
    FILTER(!isLiteral(?o))
}
"""

print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('pName', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pName', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('pName', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('pName', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('pName', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pName', 'religion')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pName', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1441'), ('pName', 'present in work')]


In [4]:
# use notable work
queryString = """
SELECT DISTINCT ?o ?pName WHERE { 

    wd:Q35610 wdt:P800 ?o.
    ?o sc:name ?pName .
}
"""

print("Results")
x = run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q2316684'), ('pName', 'canon of Sherlock Holmes')]
[('o', 'http://www.wikidata.org/entity/Q785255'), ('pName', 'The Lost World')]
2


In [5]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q2316684"
og_name = "canon of Sherlock Holmes"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"1", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 2_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_1.json
JSON object updated


## Task 2

In [6]:
# properties pointing canon of Sherlock Holmes
queryString = """
SELECT DISTINCT ?prop ?pName WHERE { 
    ?o ?prop wd:Q2316684 .
    ?prop sc:name ?pName .
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('prop', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('prop', 'http://www.wikidata.org/prop/direct/P1441'), ('pName', 'present in work')]
[('prop', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series')]
[('prop', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic")]
[('prop', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of')]
[('prop', 'http://www.wikidata.org/prop/direct/P800'), ('pName', 'notable work')]
[('prop', 'http://www.wikidata.org/prop/direct/P941'), ('pName', 'inspired by')]
[('prop', 'http://www.wikidata.org/prop/direct/P8371'), ('pName', 'references work, tradition or theory')]
9


In [7]:
# use based on
queryString = """
SELECT DISTINCT ?o ?pName WHERE { 
    ?o wdt:P144 wd:Q2316684 .
    ?o sc:name ?pName .
}
"""

print("Results")
x = run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q192837'), ('pName', 'Sherlock')]
[('o', 'http://www.wikidata.org/entity/Q1764019'), ('pName', 'Sherlock Holmes')]
[('o', 'http://www.wikidata.org/entity/Q4727821'), ('pName', 'All-Consuming Fire')]
[('o', 'http://www.wikidata.org/entity/Q247643'), ('pName', 'Elementary')]
[('o', 'http://www.wikidata.org/entity/Q9385245'), ('pName', 'Non-canonical Sherlock Holmes works')]
[('o', 'http://www.wikidata.org/entity/Q70734078'), ('pName', 'Sherlock Holmes and the Time machine')]
[('o', 'http://www.wikidata.org/entity/Q1210893'), ('pName', 'The Adventures of Sherlock Holmes and Dr. Watson')]
[('o', 'http://www.wikidata.org/entity/Q7763420'), ('pName', 'The Seven-Per-Cent Solution')]
[('o', 'http://www.wikidata.org/entity/Q200396'), ('pName', 'Sherlock Holmes')]
[('o', 'http://www.wikidata.org/entity/Q4523158'), ('pName', 'Sherlock Holmes')]
[('o', 'http://www.wikidata.org/entity/Q52204741'), ('pName', 'Miss Sherlock')]
[('o', 'http://www.wikidata

In [8]:
# get the instances
queryString = """
SELECT DISTINCT ?inst ?pName WHERE { 
    ?o wdt:P144 wd:Q2316684 ;
        wdt:P31 ?inst.
    ?inst sc:name ?pName .
}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q11424'), ('pName', 'film')]
[('inst', 'http://www.wikidata.org/entity/Q14406742'), ('pName', 'comic book series')]
[('inst', 'http://www.wikidata.org/entity/Q386724'), ('pName', 'work')]
[('inst', 'http://www.wikidata.org/entity/Q47461344'), ('pName', 'written work')]
[('inst', 'http://www.wikidata.org/entity/Q5398426'), ('pName', 'television series')]
[('inst', 'http://www.wikidata.org/entity/Q7725634'), ('pName', 'literary work')]
[('inst', 'http://www.wikidata.org/entity/Q2635894'), ('pName', 'radio drama')]
[('inst', 'http://www.wikidata.org/entity/Q3331189'), ('pName', 'version, edition, or translation')]
[('inst', 'http://www.wikidata.org/entity/Q25379'), ('pName', 'play')]
[('inst', 'http://www.wikidata.org/entity/Q24856'), ('pName', 'film series')]
[('inst', 'http://www.wikidata.org/entity/Q21191019'), ('pName', 'Japanese TV series')]
[('inst', 'http://www.wikidata.org/entity/Q1259759'), ('pName', 'miniseries')]
[('inst', 'http

In [9]:
# get the instances
queryString = """
SELECT ?inst ?pName WHERE { 
    BIND (wd:Q11424 AS ?inst).
    ?inst sc:name ?pName .
}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q11424'), ('pName', 'film')]
1


In [10]:
### insert the result of TASK 2 in the file
og_uri = "http://www.wikidata.org/entity/Q11424"
og_name = "film"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"2", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 2_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_1.json
JSON object updated


## Task 3

In [11]:
# get the instances
queryString = """
SELECT ?inst ?pName WHERE { 
    BIND (wd:Q5398426 AS ?inst).
    ?inst sc:name ?pName .
}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q5398426'), ('pName', 'television series')]
1


In [12]:
### insert the result of TASK 3 in the file
og_uri = "http://www.wikidata.org/entity/Q5398426"
og_name = "television series"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"3", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 2_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_1.json
JSON object updated


## Task 4

In [13]:
# find films and tv series related to the canon of Sherlock Holmes
queryString = """
SELECT DISTINCT ?o ?pName WHERE { 
    VALUES ?val {wd:Q11424 wd:Q5398426}.
    ?o wdt:P31 ?val;
        ?prop wd:Q2316684.
    ?o sc:name ?pName .
}
"""

print("Results")
x = run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q192837'), ('pName', 'Sherlock')]
[('o', 'http://www.wikidata.org/entity/Q247643'), ('pName', 'Elementary')]
[('o', 'http://www.wikidata.org/entity/Q604180'), ('pName', 'The Private Life of Sherlock Holmes')]
[('o', 'http://www.wikidata.org/entity/Q200396'), ('pName', 'Sherlock Holmes')]
[('o', 'http://www.wikidata.org/entity/Q4523158'), ('pName', 'Sherlock Holmes')]
[('o', 'http://www.wikidata.org/entity/Q387306'), ('pName', 'Sherlock Holmes')]
[('o', 'http://www.wikidata.org/entity/Q243643'), ('pName', 'Sherlock Holmes: A Game of Shadows')]
7


In [14]:
# see how they are related
queryString = """
SELECT ?prop ?pName COUNT(*) WHERE { 
    VALUES ?val {wd:Q11424 wd:Q5398426}.
    ?o wdt:P31 ?val;
        ?prop wd:Q2316684.
    ?prop sc:name ?pName .
}
GROUP BY ?prop ?pName
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P941'), ('pName', 'inspired by'), ('callret-2', '1')]
[('prop', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on'), ('callret-2', '6')]
2


In [15]:
# see how they are related
queryString = """
SELECT DISTINCT ?o ?oName ?prop ?pName WHERE { 
    VALUES ?val {wd:Q11424 wd:Q5398426}.
    ?o wdt:P31 ?val;
        ?prop wd:Q2316684.
    ?prop sc:name ?pName .
    ?o sc:name ?oName .
}
"""

print("Results")
x = run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q200396'), ('oName', 'Sherlock Holmes'), ('prop', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('o', 'http://www.wikidata.org/entity/Q247643'), ('oName', 'Elementary'), ('prop', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('o', 'http://www.wikidata.org/entity/Q387306'), ('oName', 'Sherlock Holmes'), ('prop', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('o', 'http://www.wikidata.org/entity/Q192837'), ('oName', 'Sherlock'), ('prop', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('o', 'http://www.wikidata.org/entity/Q4523158'), ('oName', 'Sherlock Holmes'), ('prop', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('o', 'http://www.wikidata.org/entity/Q243643'), ('oName', 'Sherlock Holmes: A Game of Shadows'), ('prop', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('o', 'http://www.wikidata.org/entity/Q60

In [16]:
# find literal properties of the films and tv series
queryString = """
SELECT DISTINCT ?prop ?pName WHERE { 
    VALUES ?val {wd:Q11424 wd:Q5398426}.
    ?o wdt:P31 ?val;
        ?prop ?x;
        ?rel wd:Q2316684.
    ?prop sc:name ?pName .
    filter(isLiteral(?x)).
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1113'), ('pName', 'number of episodes')]
[('prop', 'http://www.wikidata.org/prop/direct/P1237'), ('pName', 'Box Office Mojo film ID (former scheme)')]
[('prop', 'http://www.wikidata.org/prop/direct/P1258'), ('pName', 'Rotten Tomatoes ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1265'), ('pName', 'AlloCiné film ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1267'), ('pName', 'AlloCiné series ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1417'), ('pName', 'Encyclopædia Britannica Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1476'), ('pName', 'title')]
[('prop', 'http://www.wikidata.org/prop/direct/P1562'), ('pName', 'AllMovie title ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1712'), ('pName', 'Metacritic ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1804'), ('pName', 'DNF film ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1874'), ('pName', 'Netflix ID')]
[('prop', 'ht

In [17]:
# use publication date to get the year
queryString = """
SELECT ?o ?name (MIN(year(?x)) AS ?date) WHERE { 
    VALUES ?val {wd:Q11424 wd:Q5398426}.
    ?o wdt:P31 ?val;
        wdt:P577 ?x;
        ?rel wd:Q2316684.
    ?o sc:name ?name .
}
GROUP BY ?o ?name
"""

print("Results")
x = run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('date', '2009')]
[('o', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('date', '2011')]
[('o', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('date', '2013')]
[('o', 'http://www.wikidata.org/entity/Q604180'), ('name', 'The Private Life of Sherlock Holmes'), ('date', '1970')]
4


In [18]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    val = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= val
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"4", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 2_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_1.json
JSON object updated


## Task 5

In [19]:
# count the cast members
queryString = """
SELECT ?o ?name (COUNT(DISTINCT(?x)) AS ?cast) WHERE { 
    VALUES ?val {wd:Q11424 wd:Q5398426}.
    ?o wdt:P31 ?val;
        wdt:P161 ?x;
        ?rel wd:Q2316684.
    ?o sc:name ?name .
}
GROUP BY ?o ?name
"""

print("Results")
x = run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('cast', '15')]
[('o', 'http://www.wikidata.org/entity/Q604180'), ('name', 'The Private Life of Sherlock Holmes'), ('cast', '10')]
[('o', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('cast', '18')]
[('o', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('cast', '6')]
[('o', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('cast', '144')]
[('o', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('cast', '6')]
[('o', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('cast', '13')]
7


In [20]:
# check who worked on more films/tv series
queryString = """
SELECT ?x ?name (COUNT(DISTINCT(?o)) AS ?app) WHERE { 
    VALUES ?val {wd:Q11424 wd:Q5398426}.
    ?o wdt:P31 ?val;
        wdt:P161 ?x;
        ?rel wd:Q2316684.
    ?x sc:name ?name .
}
GROUP BY ?x ?name
HAVING (COUNT(DISTINCT ?o) > 1)
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q440926'), ('name', 'Geraldine James'), ('app', '2')]
[('x', 'http://www.wikidata.org/entity/Q190386'), ('name', 'Rachel McAdams'), ('app', '2')]
[('x', 'http://www.wikidata.org/entity/Q1102002'), ('name', 'Clive Russell'), ('app', '2')]
[('x', 'http://www.wikidata.org/entity/Q236167'), ('name', 'Kelly Reilly'), ('app', '2')]
[('x', 'http://www.wikidata.org/entity/Q165219'), ('name', 'Robert Downey Jr.'), ('app', '2')]
[('x', 'http://www.wikidata.org/entity/Q45647'), ('name', 'Eddie Marsan'), ('app', '2')]
[('x', 'http://www.wikidata.org/entity/Q160432'), ('name', 'Jude Law'), ('app', '2')]
7


In [21]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    val = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= val
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"5", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 2_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_1.json
JSON object updated


## Task 6

In [22]:
# use award received to find the Primetime Emmy Award
queryString = """
SELECT DISTINCT ?x ?name WHERE { 
    VALUES ?val {wd:Q11424 wd:Q5398426}.
    ?o wdt:P31 ?val;
        wdt:P166 ?x;
        ?rel wd:Q2316684.
    ?x sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q945887'), ('name', 'Primetime Emmy Award for Outstanding Supporting Actor in a Miniseries or a Movie')]
[('x', 'http://www.wikidata.org/entity/Q989453'), ('name', 'Primetime Emmy Award for Outstanding Lead Actor in a Miniseries or a Movie')]
[('x', 'http://www.wikidata.org/entity/Q17986500'), ('name', 'Primetime Emmy Award for Outstanding Cinematography for a Miniseries or Movie')]
[('x', 'http://www.wikidata.org/entity/Q2925698'), ('name', 'British Academy Television Award for Best Drama Series')]
[('x', 'http://www.wikidata.org/entity/Q3403230'), ('name', 'Primetime Emmy Award for Outstanding Writing for a Miniseries, Movie or a Dramatic Special')]
[('x', 'http://www.wikidata.org/entity/Q838121'), ('name', 'Peabody Awards')]
[('x', 'http://www.wikidata.org/entity/Q2925700'), ('name', 'British Academy Television Award for Best Supporting Actor')]
[('x', 'http://www.wikidata.org/entity/Q7243513'), ('name', 'Primetime Emmy Award for Outsta

In [23]:
# see the instance of Primetime Emmy Award for Outstanding Writing for a Miniseries, Movie or a Dramatic Special
queryString = """
SELECT DISTINCT ?y ?name WHERE { 
    wd:Q3403230 wdt:P31 ?y.
    ?y sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('y', 'http://www.wikidata.org/entity/Q1044427'), ('name', 'Primetime Emmy Award')]
[('y', 'http://www.wikidata.org/entity/Q38033430'), ('name', 'class of award')]
2


In [24]:
### insert the result of TASK 6 in the file
og_uri = "http://www.wikidata.org/entity/Q1044427"
og_name = "Primetime Emmy Award"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"6", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 2_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_1.json
JSON object updated


## Task 7

In [25]:
# list the Primetime Emmy Awards
queryString = """
SELECT DISTINCT ?x ?name WHERE { 
    ?x wdt:P31 wd:Q1044427.
    ?x sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q17048589'), ('name', 'Primetime Emmy Award for Outstanding Narrator')]
[('x', 'http://www.wikidata.org/entity/Q989447'), ('name', 'Primetime Emmy Award for Outstanding Lead Actress in a Miniseries or a Movie')]
[('x', 'http://www.wikidata.org/entity/Q1285504'), ('name', 'Primetime Emmy Award for Outstanding Supporting Actress in a Drama Series')]
[('x', 'http://www.wikidata.org/entity/Q2981152'), ('name', 'Primetime Emmy Award for Outstanding Guest Actress in a Comedy Series')]
[('x', 'http://www.wikidata.org/entity/Q583972'), ('name', 'Primetime Emmy Award for Outstanding Directing for a Drama Series')]
[('x', 'http://www.wikidata.org/entity/Q945887'), ('name', 'Primetime Emmy Award for Outstanding Supporting Actor in a Miniseries or a Movie')]
[('x', 'http://www.wikidata.org/entity/Q989453'), ('name', 'Primetime Emmy Award for Outstanding Lead Actor in a Miniseries or a Movie')]
[('x', 'http://www.wikidata.org/entity/Q1286639'), ('name'

In [26]:
obj = [{"uri":r[0][1],"name":r[1][1]} for r in x]
evaluation.add_result(evaluation.get_index_workflow(pt),"7", evaluation.TYPE_SET ,"uri", obj)

The index of this workflow is: 2_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_1.json
JSON object updated


## Task 8

In [27]:
# count the how many Primetime Emmy Awards each film/tv series won
queryString = """
SELECT DISTINCT ?o ?name (COUNT(?x) AS ?award) WHERE { 
    VALUES ?val {wd:Q11424 wd:Q5398426}.
    ?o wdt:P31 ?val;
        ?rel wd:Q2316684.
    OPTIONAL{?o wdt:P166 ?x.
    ?x wdt:P31 wd:Q1044427.}
    ?o sc:name ?name .
}
GROUP BY ?o ?name
"""

print("Results")
x = run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('award', '0')]
[('o', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('award', '8')]
[('o', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('award', '0')]
[('o', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('award', '0')]
[('o', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('award', '0')]
[('o', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('award', '0')]
[('o', 'http://www.wikidata.org/entity/Q604180'), ('name', 'The Private Life of Sherlock Holmes'), ('award', '0')]
7


In [28]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    val = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= val
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"8", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 2_1
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow2_1.json
JSON object updated
